In [1]:
import json
from transformers import BertTokenizer
import torch

/mnt/nas3/Alfred/espnet/tools/anaconda/envs/espnet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
json_name = f'./data/aishell_dev/50_best/bert_token/token_pll.json'

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [9]:
nbest = 30
print(nbest)

30


In [10]:
with open(json_name, 'r') as f:
    data = json.load(f)
    num = 0
    cers = []
    min_cer = 100
    best_weight = 0.0
    for num in range(100):
        weight = num * 0.01
        c = 0
        s = 0
        de = 0
        i = 0
        for d in data:
            score = torch.tensor(d['score'][:nbest])
            pll = torch.tensor(d['pll'][:nbest])

            weight_sum = (1 - weight) * score + weight * pll
            
            mask_index = torch.argmax(weight_sum).item()

            c += d['err'][mask_index][0]
            s += d['err'][mask_index][1]
            de += d['err'][mask_index][2]
            i += d['err'][mask_index][3]
        cer = (s + de + i) / (c + s + de)
        cers.append(cer)
        if (cer < min_cer):
            min_cer = cer
            best_weight = weight
    print(best_weight)    
    print(min_cer)

0.28
0.03879887601599291


In [11]:
json_name = f'./data/aishell_test/50_best/bert_token/token_pll.json'

In [12]:
with open(json_name, 'r') as f, \
    open(f'./data/aishell_test/50_best/MLM/hyp.trn', 'w') as hyp, \
    open(f'./data/aishell_test/50_best/MLM/ref.trn', 'w') as ref:
    data = json.load(f)
    weight = best_weight
    print(weight)
    c = 0
    s = 0
    de = 0
    i = 0
    for n, d in enumerate(data):
        score = torch.tensor(d['score'][:nbest])
        pll = torch.tensor(d['pll'][:nbest])

        weight_sum = (1-weight) * score + weight * pll
            
        mask_index = torch.argmax(weight_sum).item()
        max_text = list(d['text'][mask_index])


        hyp.write(f'{" ".join(max_text)} (test_{n + 1})\n')

        c += d['err'][mask_index][0]
        s += d['err'][mask_index][1]
        de += d['err'][mask_index][2]
        i += d['err'][mask_index][3]
        ref_text = list(d['ref_text'])
        ref.write(f'{" ".join(ref_text)} (test_{n + 1})\n')
    cer = (s + de + i) / (c + s + de)
    print(cer)



0.28
0.04273373741230373
